In [8]:
import numpy as np
import tiktoken
import torch
import torch.nn.functional as F
import my_classes

In [9]:
#CONSTS
MAX_SEQ_LEN    = 400
DEVICE         = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
SEED           = 25072023
NUM_EMBEDDINGS = 100264
EMBEDDING_DIM  = 1024
HIDDEN_SIZE    = 512
NUM_LAYERS     = 2        
LABELS         = 2
PARAMS_FILE    = "network_params_1.params"
torch.manual_seed(SEED)

In [10]:
def encode_data(selected_data):
        encoded_data = []
        encoder = tiktoken.get_encoding("cl100k_base") #gpt4
        for sd in selected_data:
            encoded_data.append(encoder.encode(sd))
            
        truncated_data = []
        for e in encoded_data:
            if len(e) < MAX_SEQ_LEN:
                to_add = MAX_SEQ_LEN - len(e)
                for i in range(to_add):
                    e.append(220)
                truncated_data.append(e)
            if len(e) > MAX_SEQ_LEN:
                truncated_data.append(e[:MAX_SEQ_LEN])
        return truncated_data

In [11]:
model = my_classes.Net(NUM_EMBEDDINGS, EMBEDDING_DIM, HIDDEN_SIZE, LABELS)
model.load_state_dict(torch.load(PARAMS_FILE))
model.eval()
model.to(DEVICE)

Net(
  (embedding): Embedding(100264, 1024)
  (lstm): LSTM(1024, 512)
  (lin): Linear(in_features=512, out_features=2, bias=True)
)

In [12]:
test = '''

I'm still collecting my thoughts after experiencing this film, Cillian Murphy might as well start clearing a space on his mantle for the Best Actor Oscar.

This film is a masterclass in weaving narratives and different time periods while exploring the profound depths of a man whose actions altered the world's trajectory forever, for better or worse. Nolan brings us into the complexities of Oppenheimer, and all the moral conflicts stirring within him.

Murphy's portrayal is so riveting that the long run-time became an afterthought. Robert Downey Jr also offers a great performance and Nolan's push and pull with how he uses sound design throughout is the cherry on top.

Some viewers might need a brief refresher on WWII and Cold War history, but any film lover should be happy to willingly lose themselves in this film for hours on end.

'''

In [13]:
with torch.no_grad():
    selected_data = [test]
    encoded_data = encode_data(selected_data)
    inputs = np.array(encoded_data).astype('float32')
    inputs = torch.tensor(inputs)
    inputs = inputs.type(torch.int)
    inputs  = inputs.to(DEVICE)
    inputs = inputs.T
    logits = model(inputs)
    probs = F.softmax(logits, dim=-1)
    idx = torch.argmax(probs)
    if(idx.item() == 0):
        print("negative")
    else:
        print("positive")

positive
